In [9]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
model = tf.keras.Sequential()
model.add(layers.InputLayer(input_shape=[128, 128, 1]))
init = np.ones(shape=(1, 128, 1, 512))
model.add(layers.Conv2D(512, [1, 128], 
                        strides=1, padding='valid', 
                        data_format="channels_last", 
                        activation='relu', 
                        use_bias=False, 
                        kernel_initializer=tf.constant_initializer(init),
                        kernel_constraint=tf.keras.constraints.NonNeg()))

model.compile(optimizer='adam',
              loss='mae',
              metrics=['mse'])
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 128, 1, 512)       65536     
Total params: 65,536
Trainable params: 65,536
Non-trainable params: 0
_________________________________________________________________


In [12]:
print(init.shape)

(1, 128, 1, 512)
